In [1]:
#RefLink: https://www.tensorflow.org/tutorials/distribute/keras

In [2]:
# Import TensorFlow and TensorFlow Datasets
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
tfds.disable_progress_bar()

import os

In [3]:
import tensorflow as tf


In [4]:
print(f"TensorFlow version: {tf.__version__}.")  # Keras backend
print(f"Keras version: {keras.__version__}.")
print(f"tensorflow_datasets version: {tfds.__version__}.")
print(tf.test.is_built_with_cuda())

TensorFlow version: 2.1.0.
Keras version: 2.2.4-tf.
tensorflow_datasets version: 1.2.0.
True


In [5]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

In [6]:
mnist_train

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [7]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [8]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


## Setup input pipeline

In [9]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [10]:
len(tf.config.list_physical_devices('XLA_GPU'))

0

Pixel values, which are 0-255, have to be normalized to the 0-1 range. Define this scale in a function.

In [11]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

Apply this function to the training and test data, shuffle the training data, and batch it for training. Notice we are also keeping an in-memory cache of the training data to improve performance.

In [12]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [13]:
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

## Create the model

In [14]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


## Define the callbacks

In [21]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))
log_dir = os.path.join(".", "logs")
callbacks = [
    # tf.keras.callbacks.TensorBoard(log_dir=log_dir),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

## Train and evaluate

In [22]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
    938/Unknown - 19s 20ms/step - loss: 0.1962 - accuracy: 0.9439
Learning rate for epoch 1 is 0.0010000000474974513
938/938 [==============================] - 19s 21ms/step - loss: 0.1962 - accuracy: 0.9439
Epoch 2/12
934/938 [============================>.] - ETA: 0s - loss: 0.0683 - accuracy: 0.9803
Learning rate for epoch 2 is 0.0010000000474974513
938/938 [==============================] - 8s 9ms/step - loss: 0.0683 - accuracy: 0.9803
Epoch 3/12
935/938 [============================>.] - ETA: 0s - loss: 0.0485 - accuracy: 0.9858
Learning rate for epoch 3 is 0.0010000000474974513
938/938 [==============================] - 11s 11ms/step - loss: 0.0486 - accuracy: 0.9858
Epoch 4/12
937/938 [============================>.] - ETA: 0s - loss: 0.0272 - accuracy: 0.9924
Learning rate for epoch 4 is 9.999999747378752e-05
938/938 [==============================] - 8s 8ms/step - loss: 0.0271 - accuracy: 0.9924
Epoch 5/12
931/938 [============================>.] - ETA: 0s - loss: 0